# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'traffic-signs-data/train.p'
validation_file = 'traffic-signs-data/valid.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = train['features'].shape[0]

# TODO: Number of validation examples
n_validation = valid['features'].shape[0]

# TODO: Number of testing examples.
n_test = test['features'].shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = train['features'].shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
x_labels = set([i for i in train['labels']])
n_classes = len(x_labels)

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
### Import the necessary libraries
import numpy as np
import random
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
import os
import cv2
# Visualizations will be shown in the notebook.
%matplotlib inline

### Class Distribution in the dataset

In [ ]:
## Create counter for each label to be used later
train_counts = Counter(y_train)
valid_counts = Counter(y_valid)
test_counts = Counter(y_test)

## Create histograms 
## for comparing class distribution
gs = gridspec.GridSpec(3, 1)

ax1 = plt.subplot(gs[0])
ax1.hist(y_train, bins=100)
ax1.xaxis.set_visible(False)
plt.ylabel('Training')
plt.title('Class Distribution in the Dataset')

ax2 = plt.subplot(gs[1])
ax2.hist(y_valid, bins=100)
ax2.xaxis.set_visible(False)
plt.ylabel('Validation')

ax3 = plt.subplot(gs[2])
ax3.hist(y_test, bins=100)
plt.ylabel('Test')
plt.xlabel('Classes')
plt.show()

The above plot shows the class distribution in the training, validation and test datasets.The distribution is similar across datasets, although we clearly see a skew in terms of the number of examples from each of the 43 classes in each dataset. Let's look at some of the images below.

### Sample images from the training dataset

In [ ]:
### Plot some sample images from the training dataset
plt.figure(1, figsize=(6,6))
plt.suptitle('Some sample images from the Training dataset')
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(random.choice(X_train[y_train == i*4]))
    plt.title('Class: {}'.format(i*4))
    plt.axis('off')

### Plot some special cases

Let's look at some images from the training dataset that have the minimum counts in the whole set i.e. a particular class that appears to have the minimum representative examples in the dataset.

We don't yet know how our model performs on these classes; maybe it performs well already and maybe it doesn't. We'll discuss later.

In [ ]:
## Print the under-represented classes from each set
from sklearn.utils import shuffle

## Collect the min number of counts for classes in training and validation sets
min_train_cts = min(train_counts.values())
min_valid_cts = min(valid_counts.values())

## Find the indices of these min_count classes
min_train_ind = [key for key, value in train_counts.items() if value == min_train_cts]
min_valid_ind = [key for key, value in valid_counts.items() if value == min_valid_cts]

## Plot some images
plt.figure(1, figsize=(6,6))
plt.suptitle('Some images having classes with low counts in the Training dataset')
for i in range(9):
    cl = shuffle(min_train_ind)
    plt.subplot(3,3,i+1)
    plt.imshow(random.choice(X_train[y_train == cl[0]]))
    plt.title('Class: {}'.format(cl[0]))
    plt.axis('off')

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed
def preprocess_data(n, X):
    
    ### Step 1: Normalize the data
    X = np.array(X / 255.0 - 0.5)
    
    ### Step 2: Convert RGB to grayscale
    converting_factor = np.array([[0.2989],[0.5870],[0.1140]])
    im_gray = np.zeros((n,32,32,1))
    im_gray[:,:,:,] = np.dot(X[:,:,:,], converting_factor)
    
    return im_gray

In [ ]:
X_train_pre = preprocess_data(n_train, X_train)
X_valid_pre = preprocess_data(n_validation, X_valid)
X_test_pre = preprocess_data(n_test, X_test)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf

EPOCHS = 25
BATCH_SIZE = 128

from tensorflow.contrib.layers import flatten

### Convolution, Flatten and Fully-connected layers

In [ ]:
# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
mu = 0
sigma = 0.1
    
def conv_layer(input_image, channels_in, channels_out, name="conv"):   
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal(shape=(5, 5, channels_in, channels_out), mean = mu, stddev = sigma), name="W")
        b = tf.Variable(tf.zeros(channels_out), name="B")
        conv = tf.nn.conv2d(input_image, w, strides=[1, 1, 1, 1], padding='VALID')
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')        

def flatten_op(input_image, name="flatten_operation"):
    with tf.name_scope(name):
        return flatten(input_image)   
    
def fc_layer(input_image, channels_in, channels_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal(shape=(channels_in, channels_out), mean = mu, stddev = sigma), name="W")
        b = tf.Variable(tf.zeros(channels_out), name="B")
        return tf.nn.relu(tf.matmul(input_image, w) + b)
        
    
def output_layer(input_image, channels_in, channels_out, name="output"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal(shape=(channels_in, channels_out), mean = mu, stddev = sigma), name="W")
        b = tf.Variable(tf.zeros(channels_out), name="B")
        return tf.matmul(input_image, w) + b        

In [ ]:
## Setup placeholders
with tf.name_scope("inputs"):
    inputs = tf.placeholder(tf.float32, (None, 32, 32, 1), name="inputs")
with tf.name_scope("labels"):
    labels = tf.placeholder(tf.int32, (None), name="labels")
with tf.name_scope("keep_prob"):
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
with tf.name_scope("one_hot_y"):
    one_hot_y = tf.one_hot(labels, n_classes, name="one_hot_y")

In [ ]:
## Build the model   
# Layer 1: Convolutional. Input = 32x32x1. Output = 14x14x6.
conv1 = conv_layer(inputs, 1, 6, "conv1")

# Layer 2: Convolutional. Output = 5x5x16.
conv2 = conv_layer(conv1, 6, 16, "conv2")

# Flatten. Input = 5x5x16. Output = 400.
flattened   = flatten_op(conv2)
    
# Layer 3: Fully Connected. Input = 400. Output = 120.
fc1 = fc_layer(flattened, 400, 120, "fc1")
fc1 = tf.nn.dropout(fc1, keep_prob)

# Layer 4: Fully Connected. Input = 120. Output = 84.
fc2 = fc_layer(fc1, 120, 84, "fc2")
fc2 = tf.nn.dropout(fc2, keep_prob)

# Output Layer : Fully Connected. Input = 84. Output = 43.
logits = output_layer(fc2, 84, n_classes)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

rate = 0.001
    
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits, name="cross_entropy")
with tf.name_scope("loss_operation"):
    loss_operation = tf.reduce_mean(cross_entropy, name="loss_operation")
    
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate = rate, name="optimizer")
with tf.name_scope("training_operation"):
    training_operation = optimizer.minimize(loss_operation)

In [ ]:
with tf.name_scope("predictions"):
    predictions = tf.argmax(tf.nn.softmax(logits), 1, name="predictions")
with tf.name_scope("probability"):
    probability = tf.nn.softmax(logits, name="probability")

with tf.name_scope("correct_prediction"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1), name="correct_prediction")
with tf.name_scope("accuracy_operation"):
    accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy_operation")
    
tf.summary.scalar("loss", loss_operation) 
tf.summary.scalar("accuracy", accuracy_operation)

saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={inputs: batch_x, labels: batch_y, keep_prob: 1.})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
# Export the nodes
from collections import namedtuple
export_nodes = ["inputs", "labels", "keep_prob", "loss_operation", "predictions", "accuracy_operation"]
Graph = namedtuple('Graph', export_nodes)
local_dict = locals()
graph = Graph(*[local_dict[each] for each in export_nodes])

In [ ]:
with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter("./logs/1", sess.graph)
    
    num_examples = len(X_train_pre)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train_pre, y_train = shuffle(X_train_pre, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_pre[offset:end], y_train[offset:end]
            if (offset % 5) == 0:
                s = sess.run(merged_summary, feed_dict={inputs: batch_x, labels: batch_y, keep_prob: 0.75})
                file_writer.add_summary(s, offset)                
            sess.run(training_operation, feed_dict={inputs: batch_x, labels: batch_y, keep_prob: 0.75})
            
        validation_accuracy = evaluate(X_valid_pre, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')

    print("Model saved")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test_pre, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
X_from_web = []
for i in range(5):
    im = mpimg.imread('traffic_signs_web/0013{}.ppm'.format(i))
    im = cv2.resize(im, (32,32), interpolation = cv2.INTER_CUBIC)
    X_from_web.append(im)

for i in range(5):
    plt.subplot(2, 3, i+1)
    plt.imshow(X_from_web[i])
    plt.axis('off')
plt.suptitle('Five German Traffic signs from the web')
plt.show()
print("Image data shape =", np.shape(X_from_web[0]))

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

#no. of images in X_web
n_from_web = len(X_from_web)

# Initialize an empty list for storing processed images
X_web_processed = []

# Run web images through the pre-processing pipeline
for i in range(len(X_from_web)):
    X_web_processed.append(preprocess_data(n_from_web, X_from_web[i])[0])

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    preds = sess.run(predictions, feed_dict={inputs: X_web_processed, keep_prob: 1.})
    print("Predictions = {}".format(preds))

In [ ]:
y_labels_web = np.array([36,3,14,13,11])

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    accuracy = sess.run(accuracy_operation, feed_dict={inputs: X_web_processed, labels: y_labels_web, keep_prob: 1.})
    print("Accuracy = {:.4f}".format(accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    top_k = sess.run(tf.nn.top_k(probability, k=5), feed_dict={inputs: X_web_processed, keep_prob: 1.})
    print("Top 5 Softmax probabilities = {}".format(top_k))

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=2 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    image_input_pre = preprocess_data(1, image_input)
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint('.'))
        activation = tf_activation.eval(session=sess,feed_dict={inputs : image_input_pre})
        featuremaps = activation.shape[3]
        plt.figure(plt_num, figsize=(15,15))
        for featuremap in range(featuremaps):
            plt.subplot(6,8, featuremap+2) # sets the number of feature maps to show on each row and column
            plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
            if activation_min != -1 & activation_max != -1:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
            elif activation_max != -1:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
            elif activation_min !=-1:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
            else:
                plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")
        plt.subplot(6,8,1)
        plt.title('Original Image')
        plt.imshow(image_input)

In [ ]:
outputFeatureMap(X_from_web[0], conv1)